<a href="https://colab.research.google.com/github/Neloy262/Reinforcement_Learning/blob/master/Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import gym
from IPython import display

In [158]:
env=gym.make('BreakoutDeterministic-v4')
rgb_weights = [0.2989, 0.5870, 0.1140]
print(env.action_space.n)

4


In [91]:
def preprocess(img):
  img=np.dot(img[...,:3], rgb_weights)
  img=img[::2,::2]
  img=img[16:98]
  return img


In [155]:
state=env.reset()
state=np.array(state)
action=env.action_space.sample()
state1, reward, done, info=env.step(action)
state2, reward, done, info=env.step(action)
state3, reward, done, info=env.step(action)
state4, reward, done, info=env.step(action)
state1=preprocess(state1)
state2=preprocess(state2)
state3=preprocess(state3)
state4=preprocess(state4)
arr=np.array([state1,state2,state3,state4])
newArr=np.stack(arr,axis=2)
print(newArr.shape)
# plt.imshow(newArr[1], cmap=plt.get_cmap("gray"))

(82, 80, 4)


In [113]:
state=env.reset()
done=False
while not done:
  #  plt.imshow(env.render(mode='rgb_array'))
  #  display.display(plt.gcf())    
  #  display.clear_output(wait=True)
   action=env.action_space.sample()
   state, reward, done, info=env.step(action) # take a random action
   print(done)
display.set_matplotlib_close
env.close()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [159]:
class DQNAgent:
    def __init__(self,state_size,action_size):
        self.state_size=state_size
        self.action_size=action_size
        self.memory=deque(maxlen=2000)
        self.gamma=0.95
        self.epsilon=1.0
        self.epsilon_decay=0.995
        self.epsilon_min=0.01
        self.learning_rate=0.001
        self.model=self.build_model()
        self.target_model=self.build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.target_update_counter=0
        
    def build_model(self):
        model=Sequential()
        model.add(Conv2D(filters=16,kernel_size=(8,8),strides=4,activation='relu',input_shape=(82,80,4)))
        model.add(Conv2D(filters=32,kernel_size=(4,4),strides=2,activation='relu',input_shape=(82,80,4)))
        model.add(Flatten())
        model.add(Dense(units=256,activation='relu'))
        model.add(Dense(units=env.action_space.n,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    
    def act(self,state):
#         if np.random.rand() <= self.epsilon:
#             return random.randrange(self.action_size)
        act_values=self.model.predict(state)
        return np.argmax(act_values[0])
        
        
    def replay(self,batch_size):
        minibatch=random.sample(self.memory,batch_size)
        
        for state,action,reward,next_state,done in minibatch:
            target=reward
            
            if not done:
                target=reward+self.gamma*np.amax(self.target_model.predict(next_state)[0])
            target_f=self.model.predict(state)
            target_f[0][action]=target

            self.model.fit(state,target_f,epochs=1,verbose=0)
            self.target_update_counter=self.target_update_counter+1
            
            if self.target_update_counter>=10:
                self.update_target_network()
                self.target_update_counter=0
            
        if self.epsilon>self.epsilon_min:
            self.epsilon*=self.epsilon_decay
    
    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def load(self,name):
        self.model.load_weights(name)
        
        
    def save(self,name):
        self.model.save_weights(name)